In [4]:
import sys
sys.path.append('..')

import numpy as np
import pickle
import matplotlib.pyplot as plt
import os
import time

import torch

from utils import *
TensorType = torch.DoubleTensor
from nmgp_dsvi import *


In [8]:
data = "HCP"
data_file = "data.pickle"
if not os.path.exists("../../res/{}".format(data)):
    os.mkdir("../../res/{}".format(data))

# Upload Data
with open("../../data/{}/{}".format(data, data_file), "rb") as res:
    X_list, Y_list, Xt_list, Yt_list = pickle.load(res)

n_dims = len(X_list)

# convert data format
X_train_list = [x[:, None] for x in X_list]
X_test_list = [x[:, None] for x in Xt_list]
X_train_vec = np.concatenate(X_train_list)
X_test_vec = np.concatenate(X_test_list)
Y_train_list = [y[:, None] for y in Y_list]
Y_test_list = [y[:, None] for y in Yt_list]
Y_train_vec = np.concatenate(Y_train_list)
Y_test_vec = np.concatenate(Y_test_list)
train_index = np.concatenate([np.ones_like(Y_train_list[i])*i for i in range(n_dims)]).astype(int)
test_index = np.concatenate([np.ones_like(Y_test_list[i])*i for i in range(n_dims)]).astype(int)

t_max = np.max([np.max(np.concatenate(X_list)), np.max(np.concatenate(Xt_list))])

do_plot_raw_data = True

if do_plot_raw_data:
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.set_title('Output 1')
    ax1.plot(X_train_list[0], Y_train_list[0],'kx',mew=1.5,label='Train set')
    ax1.plot(X_test_list[0], Y_test_list[0],'rx',mew=1.5,label='Test set')
    ax1.legend()
    plt.show()

In [7]:
# Sparse GPR
import GPy

K = GPy.kern.Exponential(1)
m1 = GPy.models.SparseGPRegression(X_train_list[0], Y_train_list[0], kernel=K.copy(), num_inducing=100)
m1.optimize(messages=True)
test_index = np.concatenate([i*np.ones_like(X_test_list[i]) for i in range(n_dims)]).astype(int)
est_Y_test = m1.predict(X_test_list[0])[0]

print(m1)

rmse_test = np.sqrt(np.mean((est_Y_test - np.vstack(Y_test_list)) ** 2))
print(rmse_test)
# fig = plt.figure()
# ax1 = fig.add_subplot(111)
# ax1.set_title('Output 1')
# ax1.plot(grids, est_Y_grid,'kx',mew=1.5, label='Prediction set')
# ax1.legend()
# plt.show()

In [6]:
# Sparse Corregionalization Model
import GPy

K = GPy.kern.Exponential(1)
icm = GPy.util.multioutput.ICM(input_dim=1,num_outputs=n_dims,kernel=K)
m2 = GPy.models.SparseGPCoregionalizedRegression(X_train_list,Y_train_list,kernel=icm,num_inducing=100)
m2['.*Exponential.var'].constrain_fixed(1.) #For this kernel, B.kappa encodes the variance now.
print("start")
print(m2.log_likelihood())
m2.optimize(messages=True)

ts = time.time()
Xt_vecs = np.vstack([np.hstack([X_test_list[i], np.ones_like(X_test_list[i])*i]) for i in range(n_dims)])
noise_dictt = {'output_index':Xt_vecs[:,1].astype(int)}
est_Y_test = m2.predict(Xt_vecs, Y_metadata=noise_dictt)[0]

print(m2)

rmse_test = np.sqrt(np.mean((est_Y_test - np.vstack(Y_test_list)) ** 2))
print(rmse_test)
print(time.time() - ts)